# Analysis of inter-embryo variability using Kullback-Leibler divergence across genetically identical embryos

### *Gene expression variance and invariance across sibling embryos. We devised a framework to quantify gene expression differences across individual embryos. Our framework relies on estimating probability distributions of gene expression across single cells for each embryo and applying an information-theoretic approach to quantify the differences between those distributions.*

In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl


from scipy.special import rel_entr, kl_div
from scipy.special import softmax
from scipy.stats import entropy
from scipy import stats

from functools import reduce

## Uploading adata and get the KLD dataframe for a single timepoint

### We prepare the adata object:
* Choose a single timepoint for the analysis
* Find cell clusters with at least 80 cells per embryo
* If we want to compute the analysis for the entire embryo, we call the function 'make_columns' with the annotation 'cell_type_1'. Otherwise, we call the function with the desired cell clusters, and the column in which the cell cluster labels can be found


In [18]:
# function: subsets adata for a timepoint
# arguments: adata object and desired timepoint
# output: subset adata object
    
def myadata(adata, timepoint):
    myadata = adata[adata.obs.timepoint == timepoint].copy()
    
    return myadata

In [27]:
def get_cell_clusters(adata, celltypecolumn = 'ontology_class'):
    myfish = adata.obs.fish.unique()
    celltypes = []

    for fish in myfish:
            counts = adata.obs[adata.obs.fish == fish][celltypecolumn].value_counts()
            l = counts[counts > 80].index.tolist()
            celltypes += l
            
    alltypes = set([x for x in celltypes if celltypes.count(x) == 4])
    return list(alltypes)

In [19]:
# function: some preprocessing, go from adata object to input for 
# input: adata object, and either a list of cell types with the cell type column name, or 'cell_type_1' if whole embryo
# output: adata with new 'Cells_Annotations' column, the cell type(s) of interest for analysis

def make_columns(adata, annotations = ['cell_type_1'], celltypecolumn = 'ontology_class'):
    
    if annotations[0] == 'cell_type_1':
        newadata = adata.copy()
        newadata.obs['Cells_Annotations'] = 'cell_type_1'
        
    else:
        newadata = adata[adata.obs[celltypecolumn].isin(annotations)].copy()
        newadata.obs['Cells_Annotations'] = newadata.obs[celltypecolumn].copy()
        
    
    return newadata


## *Example code:*
* Upload adata object
* Chose timepoint and get subset adata ('myadata' function)
* Find potential cell clusters to run analysis with at least 80 cells per embryo ('get_cell_clusters' function)
* Now we have our example_adata, ready to run the inter-individual analysis

In [15]:
adata = sc.read_h5ad('/mnt/ibm_lg/alejandro/danio-atlas/zebrahub/final_objects/v4/zf_atlas_full_v4_release.h5ad')
adata12hpf = myadata(adata, '12hpf')
get_cell_clusters(adata12hpf, 'zebrafish_anatomy_ontology_class')
example_adata = make_columns(adata12hpf, ['periderm'], 'zebrafish_anatomy_ontology_class')

## Inter-individual analysis (KLD)


#### **Computes the median pairwise Kullback-Leibler divergence across genetically identical embryos for a single cell cluster and single timepoint**

* Takes the log normalized counts grouped by embryo for each gene
* Excludes genes that have less than 10 non-zero counts for each embryo
* Fits a probability density function for the expression of each gene (for each individual embryo per time point) using a Gaussian kernel
* Uses the Kullback-Leibler divergence to estimate, for each gene and per stage, all pairwise divergences between embryo gene count distributions

In [6]:
# function: calculation of KL for two numpy arrays
# arguments: two numpy arrays
# output: KL score (float)

def kl_divergence(fish1gene = [],  fish2gene = []):
    
    
    pkde = stats.gaussian_kde(np.array(fish1gene))
    qkde = stats.gaussian_kde(np.array(fish2gene))
    maxval = np.max([np.max(fish1gene), np.max(fish2gene)])
    minval = np.min([np.min(fish1gene), np.min(fish2gene)])
    
    ### check gaussian_kde, no parameter that could change results (ie. range)
    offset = 1
    newx = np.linspace(minval-offset, maxval+offset,1000)
    
    p = pkde.pdf(newx)
    q = qkde.pdf(newx)
    
    newp = np.where(p< 1e-9, 1e-9, p)
    newq = np.where(q< 1e-9, 1e-9, q)

    
    kl_pair = stats.entropy(newp, newq)
    return kl_pair

Gets the arrays of RNA counts for the gene/cell type for a pair of fish

In [7]:
# function: helper to make the matrix, gets the specific arrays and sends to kl function
# arguments: matrix of counts (sparse), gene, cell type, two fish (str), layer (log/scaled str), obs and var from adata as pd.DF
# output: kl divergence score for two arrays (calls kl function)

def kl_fishpair_genecelltype2(matrix = [[],[]], gene = 'phtf2', cluster = 'Somite', fish1 = '44', fish2 = '46', obs=pd.DataFrame, var=pd.DataFrame):
       
 
    row1 = obs['fish'].loc[lambda x: x==fish1].index.tolist()
    row2 = obs['fish'].loc[lambda x: x==fish2].index.tolist()
    
    myindex = int(var.index[var['index']==gene].tolist()[0])
    

    q = np.asarray(matrix.tocsr()[row1, myindex].todense()).flatten()
    p = np.asarray(matrix.tocsr()[row2, myindex].todense()).flatten()
    
    if (np.sum(p > 0) < 10) or (np.sum(q > 0) < 10):
        return -1
    else:
        return kl_divergence(q, p)

Makes a matrix of pairwise fish

In [8]:
# function: makes a matrix with the pairwise KL values
# arguments: takes in a matrix of counts (sparse), a gene, cell type, list of fish (str), layer (log/scaled str), obs and var from adata as pd.DF
# output: a matrix with all pairwise KL scores for the fish

def kl_fish_matrix2(matrix = [[],[]], gene = 'phtf2', cluster = 'Somite', fish = [], obs =pd.DataFrame, var=pd.DataFrame):

    fishcols = []
    
    for f1 in fish:
        f1row = []
        for f2 in fish:
            value = kl_fishpair_genecelltype2(matrix, gene, cluster, f1, f2, obs, var)
            
            if (value == -1):
                return -1
            f1row.append(value)
        fishcols.append(f1row)      

    return fishcols 

Summarizes the pairwise fish matrix (excluding the diagonal elements, compared to self) by the numpy method (i.e. np.mean) specified

In [9]:
# function: calculates summary metric of kl matrix pairwise scores, not including the diagonal elements
# arguments: matrix with kl pairwise scores, summary method metric (numpy)
# output: single summary number of matrix (float)

def kl_matrix_summary3(myfishmatrix, method = np.max):
    testmatrix = np.array(myfishmatrix)
    numzeros = len(testmatrix)
    newx = np.sort(testmatrix, axis=None)
    
    allelem = newx[numzeros:]

    

    return method(allelem)

Function that calls the process for a specific gene and builds the KLD dataframe

In [10]:
# function: return a dataframe with the desired KL metric summaries for specified genes, for a certain cell type
# arguments: adata object, list of genes, cell type (str), list of fish (str), layer (log/scaled str), list of np summary methods (mean/median, etc)
# output: pd dataframe, with columns gene, cluster (cell type), and summary methods


def KL_metrics_genecluster(adata = [], gene = [], cluster = 'Somite', myfish = [], method = [], timepoint = '12hpf'):
    
    myadata = adata[adata.obs['Cells_Annotations'] == cluster].copy()
    a_var = myadata.var.copy().reset_index()
    a_obs = myadata.obs.copy().reset_index()
    newmatrix = myadata.X.copy()
    
    mygenes = []
    generow = {}
    for g in gene:
        mymatrix = kl_fish_matrix2(newmatrix, g, cluster, myfish, a_obs, a_var)
        
        if mymatrix == -1:
            continue
        
        generow = {'gene': g, 'cluster': cluster}
        for m in method:
            stat = kl_matrix_summary3(mymatrix, m)
            
            generow[m.__name__] = stat

        mygenes.append(generow)
    
       
    bigframe = pd.DataFrame.from_dict(mygenes)
    bigframe['timepoint'] = timepoint
    return bigframe

We call this function to compute the KL dataframe

In [11]:
# function: creates a dataframe with KL metrics, per gene, grouped by cell type
# arguments: adata, list of cell types/annotations, out_path as str, title as string 
# output: returns a dataframe with KL metrics per gene, per cell type

def KLdf(adata = [], annotations = ['Hindbrain'], myfish = [], timepoint = 'hpf'):
    KLframes = []
    mymetrics = [np.max, np.min, np.median, np.std, np.mean]
    mygenes = np.asarray(adata.var_names)
    for a in annotations:
        miniframe = KL_metrics_genecluster(adata, mygenes, a, myfish, mymetrics, timepoint)
        KLframes.append(miniframe)


    KL_metrics = pd.concat(KLframes, axis=0)
    
    return KL_metrics

### Creates and saves the KLD dataframe
#### *This is the function we call to create and save the KLD dataframe. We input the singular timepoint adata object and the cell types we want to run, and include the title and path to place to save the dataframe*

In [12]:
# function: exports a csv dataframe with KL metrics, per gene, grouped by cell type
# arguments: adata, list of cell types/annotations, out_path as str, title as string 
# output: saves a CSV dataframe with KL metrics per gene, per cell type at the out_path

def saveKLdf(adata = [], annotations = ['Hindbrain'], out_path = '', save_title = 'date', timepoint = 'hpf'):
    myfish = adata.obs['fish'].unique()
    KL_metrics = KLdf(adata, annotations, myfish, timepoint)
    

    KL_metrics.to_csv(out_path + save_title + 'Inter.csv')
    
    return KL_metrics


## *Example code:*

* We use our 'example_adata = make_columns(adata12hpf, ['periderm'], 'zebrafish_anatomy_ontology_class')' from earlier
* The timepoint variable should include 'hpf' hours post fertilization for later processing
* We define the path to save and the title of the csv, it will be saved as out_path + save_title + 'Inter.csv'

In [30]:
myout_path = '/mnt/ibm_lg/sarah.ancheta/ZF_atlas/intra_inter_v4/'
mysave_title = 'periderm_test'

In [ ]:
saveKLdf(adata = example_adata, annotations = ['periderm'], out_path = myout_path, save_title = mysave_title, timepoint = '12hpf')

## Null model for median KL divergences through embryo ID randomization

#### *Gives us the expected median KL values if the differences between embryos were due to chance alone* 

* To establish a negative control and to gain a baseline for the difference in KL divergence between embryos (for a specific gene), we performed a bootstrap-like method
* Each cell has an embryo-specific ID, which we randomly reassigned
* We then repeated the inter-embryo diversity analysis process as described above
* We repeated this process n = 20 times

In [ ]:
# function: saves a csv pandas DF with intrafish metrics, with numrandomstates different random scrambled fish states
# arguments: adata, cell types list of str, out_path (str) and title to save (str)
# output: csv with scrambled fish, same as intrafish DF but with added column of the random state

def scrambledInterFish(adata = [], annotations = ['Hindbrain'], numrandomstates = 20, timepoint = 'hpf'):
    
    scrambleDFframes = []
    myfish = adata.obs['fish'].unique()
    
    for i in range(numrandomstates):
        adata1 = adata.copy()
        adata1.obs['fish'] = np.random.RandomState(seed=i).permutation(adata1.obs['fish'].values)
        state = str(i)
        
        fishscramble = KLdf(adata1, annotations, myfish, timepoint)
        fishscramble['Random_State']='mixed_fish' + state
        scrambleDFframes.append(fishscramble)
        
        
    scrambleInter = pd.concat(scrambleDFframes, axis=0)
    
    
    return scrambleInter


In [ ]:
# function: saves scrambled (null) df with KL analysis
# arguments: adata, list of cell types/annotations, number of randomzied iterations, out_path as str, title as string 
# output: returns a dataframe with KL metrics per gene, per cell type, with random state column (which iteration of random)

def save_scrambledInter(adata = [], annotations = ['Hindbrain'], numrandomstates = 20, out_path = '', save_title = 'date', timepoint = 'hpf'):
    
    mydf = scrambledInterFish(adata, annotations, numrandomstates, timepoint)
    mydf.to_csv(out_path + save_title + 'null_scramble.csv')
    
    return mydf

## *Example code:*

* Same as above for the KLD dataframe, but now include the number of random iterations
* We use our 'example_adata = make_columns(adata12hpf, ['periderm'], 'zebrafish_anatomy_ontology_class')' from earlier
* The timepoint variable should include 'hpf' hours post fertilization for later processing
* We define the path to save and the title of the csv, it will be saved as out_path + save_title + 'Inter.csv'

In [24]:
myout_path = '/mnt/ibm_lg/sarah.ancheta/ZF_atlas/intra_inter_v4/'
mysave_title = 'periderm_test_null_randomized'

In [ ]:
save_scrambledInter(adata = example_adata, annotations = ['periderm'], numrandomstates = 20, out_path = myout_path, save_title = mysave_title, timepoint = '12hpf')